# Introducción

En esta notebook se analizará la serie de tiempo de BTC. El desarrollo consta de tres partes: la primera prepara el dataset, la segunda aplica modelos de predicción y la tercera analiza estacionariedad tanto para la serie de tiempo como para los residuos de los modelos

# 1) Preparación previa

## Carga de librerías

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib inline

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.graphics.tsaplots import plot_predict
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

from scipy import stats
from statistics import mode

from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Se debe instalar pmdarima 
from pmdarima import auto_arima #!pip install pmdarima

# Se debe instalar prophet
from prophet import Prophet #!pip install prophet
from prophet.diagnostics import cross_validation
import itertools
from prophet.diagnostics import performance_metrics

import warnings
warnings.filterwarnings('ignore')
import logging

## Lectura y armado del dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Agustin-Bulzomi/Projects/main/Programming/Digital%20House%20(Python)/Support%20Files/Final%20Project/coin_Bitcoin.csv', delimiter = ',')

Se realizan las modificaciones del dataset pertinentes para el análisis de series de tiempo

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.index = pd.PeriodIndex(df.Date, freq = 'D')
df.head()

Se agrega la columna Time index, necesaria para algunos modelos futuros

In [ ]:
df['timeIndex'] = pd.Series(np.arange(len(df['Close'])), index = df.index)

df.head()

In [ ]:
df.tail()

#### Se crean dummies de los meses, que serán utilizadas luego en un modelo que analiza estacionalidad

In [ ]:
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
dummies_mes = pd.get_dummies(df['Month'], drop_first = True, prefix = 'Month')
df = df.join(dummies_mes)
df.sample(10)

## División de Train y Test

In [ ]:
# Se procede con el tradicional 90-10, recomendado para time series y/o datasets muy grandes
df_train, df_test = train_test_split(df, test_size=0.1, shuffle=False)

print("train shape", df_train.shape)
print("test shape", df_test.shape)

#### Ploteo de los dos datasets obtenidos:

In [ ]:
pd.plotting.register_matplotlib_converters()
f, ax = plt.subplots(figsize = (14,5))
df_train.plot(kind = 'line', x = 'Date', y = 'Close', color = 'blue', label = "Train", ax = ax)
df_test.plot(kind = 'line', x = 'Date', y = 'Close', color = 'red', label = "Test", ax = ax)
ax.legend(loc = 'upper left')
plt.title("Rango para Train y para Test")
plt.show()

Visualmente ya se puede ver que es prácticamente imposible que un modelo de predicción estime el ascenso que tuvo el BTC en el último medio año, por lo que el split 90-10 tradicional no va a permitir evaluar modelos acertadamente. Se procede a hacer un split manual para analizar solo el último mes disponible de 2021 (febrero) como test.

In [ ]:
train_size_date = df[df['Date'] <= (df['Date'].max() - pd.DateOffset(days=31))].shape[0]
df_train, df_test = train_test_split(df, train_size=train_size_date, shuffle=False)

print("train shape", df_train.shape)
print("test shape", df_test.shape)

In [ ]:
df_test.head()

#### Ploteo de los dos datasets obtenidos:

In [ ]:
pd.plotting.register_matplotlib_converters()
f, ax = plt.subplots(figsize = (14,5))
df_train.plot(kind = 'line', x = 'Date', y = 'Close', color = 'blue', label = "Train", ax = ax)
df_test.plot(kind = 'line', x = 'Date', y = 'Close', color = 'red', label = "Test", ax = ax)
ax.legend(loc = 'upper left')
plt.title("Rango para Train y para Test")
plt.show()

## Generación de la serie en escala logarítmica

In [ ]:
df_train['log_value'] = np.log(df_train['Close'])
df_test['log_value'] = np.log(df_test['Close'])

#### Ploteo del Target y Test:

In [ ]:
pd.plotting.register_matplotlib_converters()
f, ax = plt.subplots(figsize = (14,5))
df_train.plot(kind = 'line', x = 'Date', y = 'log_value', color = 'blue', label = "Train", ax = ax)
df_test.plot(kind = 'line', x = 'Date', y = 'log_value', color = 'red', label = "Test", ax = ax)
ax.legend(loc = 'upper left')
plt.title("Rango para Train y para Test")
plt.show()

# 2) Modelos

Se utilizará una plétora de herramientas y recursos para analizar las series de tiempo y sus implicancias. En cada paso se irá visualizando los resultados y almacenando su información para, al final de la notebook, compararlos

#### Se define una función para calcular el RMSE:

In [ ]:
def RMSE(actual, predicted):
  mse = (predicted - actual) ** 2
  rmse = np.sqrt(mse.sum() / mse.count())
  return rmse

#### Se define una función para calcular el MAPE:

In [ ]:
def MAPE(actual, predicted): 
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

#### Se define una función para crear los gráficos de cada modelo:

In [ ]:
def plot_time_series(df_train, df_test, model_name, series = 'Close'):
  fig, axes = plt.subplots(1,2, figsize = (16, 6))

  df_train.plot(kind = "line", y = [series, model_name], ax = axes[0])
  axes[0].set_title("Train Data", size = 16)
  axes[0].set_xlabel("Year", size = 14)

  df_test.plot(kind = "line", y = [series, model_name], ax = axes[1])
  axes[1].set_title("Test Data", size = 16)
  axes[1].set_xlabel("Date", size = 14)
  # Por el salto de mes que se da de enero a febrero, el código plotea los x ticks con variedad de formatos. Se lo modifica:
  date_format = mdates.DateFormatter('%b-%d')
  axes[1].xaxis.set_major_formatter(date_format)
  weekday_locator = mdates.WeekdayLocator(byweekday = mdates.MO)
  axes[1].xaxis.set_major_locator(weekday_locator)

  # Se agregan el RMSE y el MAPE debajo de los plots
  rmse = RMSE(df_test[series], df_test[model_name])
  mape = MAPE(df_test[series], df_test[model_name])
  rmse_mape_text = f"RMSE = {rmse:.2f} | MAPE = {mape:.2f}%"
  plt.text(0.5, 0.0, rmse_mape_text, ha = 'center', transform = fig.transFigure, fontsize = 14)
  
  #Se agrega el título del plot
  formatted_model_name = model_name.replace("_", " ").title()
  plt.suptitle(f"Predicción del precio de BTC con {formatted_model_name}", size = 18, y = 1.02)
  
  plt.show()

## a) Mean

#### Se aplica el modelo de media constante a train y test:

In [ ]:
# Se calcula el promedio:
model_mean_pred = df_train['Close'].mean()

# La predicción es fija y es la misma para el set de testeo y de entrenamiento:
df_train['mean'] = model_mean_pred
df_test['mean'] = model_mean_pred

#### Ploteo de las predicciones vs la serie real y cálculo de RMSE y MAPE:

In [ ]:
plot_time_series(df_train, df_test, 'mean')

#### Se guardan los resultados en un DataFrame:

El mismo será reutilizado para almacenar los resultados de los distintos modelos a utilizar

In [ ]:
df_results = pd.DataFrame(columns = ["Model", "RMSE","MAPE"])
df_results.loc[0, "Model"] = "Mean"
df_results.loc[0, "RMSE"] = round(RMSE(df_test['Close'], df_test['mean']),1)
df_results.loc[0, "MAPE"] = round(MAPE(df_test['Close'], df_test['mean']),1)
df_results

## b) Random Walk

Se crea el shift de target en train:

In [ ]:
df_train['close_shift'] = df_train['Close'].shift()
# La primera observación va a quedar en nan, por lo que se reemplaza por el valor siguente:
df_train['close_shift'].fillna(method = 'bfill', inplace = True)
df_train.head()

Se crea el shift de target en test:

In [ ]:
df_test['close_shift'] = df_test['Close'].shift()
# Se puede reemplazar el primer nan con el último valor del set de entrenamiento:
df_test.iloc[0,26] = df_train.iloc[-1,0]
df_test.head()

Lag de un período:

In [ ]:
df_train.plot(kind = 'scatter', y = 'Close', x = 'close_shift', s = 50);

Diferencias entre Target y el lag:

In [ ]:
df_train['close_diff'] = df_train['Close'] - df_train['close_shift']
df_train['close_diff'].plot()

#### Ploteo de las predicciones vs la serie real y cálculo de RMSE y MAPE:

In [ ]:
df_train['random_walk'] = df_train['close_shift']
df_test['random_walk'] = pd.Series(df_train['Close'][-1], index = df_test.index)

In [ ]:
plot_time_series(df_train, df_test, 'random_walk')

#### Se almacenan los valores de RMSE y MAPE

In [ ]:
df_results.loc[1, "Model"] = "Random Walk"
df_results.loc[1, "RMSE"] = round(RMSE(df_test['Close'], df_test['random_walk']),1)
df_results.loc[1, "MAPE"] = round(MAPE(df_test['Close'], df_test['random_walk']),1)
df_results

## c) Linear Trend

#### Se crea una columna en train con el predict:

In [ ]:
model_linear = smf.ols('Close ~	timeIndex', data = df_train).fit()

df_train['linear_trend'] = model_linear.predict(df_train['timeIndex'])
df_test['linear_trend'] = model_linear.predict(df_test['timeIndex'])

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
plot_time_series(df_train, df_test, 'linear_trend')

#### Se almacenan los valores de RMSE y MAPE

In [ ]:
df_results.loc[2, "Model"] = "Linear Trend"
df_results.loc[2, "RMSE"] = round(RMSE(df_test['Close'], df_test['linear_trend']),1)
df_results.loc[2, "MAPE"] = round(MAPE(df_test['Close'], df_test['linear_trend']),1)
df_results

## d) Back Log Transformation + Linear Trend

Se fitea el modelo Linear Trend con escala logarítmica

In [ ]:
model_log = smf.ols('log_value ~ timeIndex', data = df_train).fit()

In [ ]:
model_log.summary()

In [ ]:
df_train['log_linear'] = model_log.predict(df_train[['timeIndex']])
df_test['log_linear'] = model_log.predict(df_test[['timeIndex']])

Se invierte la escala logarítmica del modelo anterior

In [ ]:
df_train['back_linear'] = np.exp(df_train['log_linear'])
df_test['back_linear'] = np.exp(df_test['log_linear'])

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
plot_time_series(df_train, df_test, 'back_linear')

#### Se almacenan los valores de RMSE y MAPE

In [ ]:
df_results.loc[3, "Model"] = "Back Log Linear"
df_results.loc[3, "RMSE"] = round(RMSE(df_test['Close'], df_test['back_linear']),1)
df_results.loc[3, "MAPE"] = round(MAPE(df_test['Close'], df_test['back_linear']),1)
df_results

## e) Back Log Transformation + Linear Trend + Estacionalidad

En la tercera sección de esta notebook se utilizarán algunas herramientas para analizar la estacionalidad de la serie. Sin embargo, igualmente se analiza un caso de modelo con agregado de estacionalidad para ver si aporta al resultado.

#### Creación del modelo con dummies

In [ ]:
log_linear_est = smf.ols('log_value ~ timeIndex + Month_2 + Month_3 + Month_4 + Month_5 + Month_6 + Month_7 + Month_8 + Month_9 + Month_11 + Month_12', data = df_train).fit()

df_train['log_linear_est'] = log_linear_est.predict(df_train[['timeIndex', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6','Month_7', 'Month_8', 'Month_9','Month_10','Month_11','Month_12']])

df_test['log_linear_est'] = log_linear_est.predict(df_test[['timeIndex', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6','Month_7', 'Month_8', 'Month_9','Month_10','Month_11','Month_12']])

Se invierte la escala logarítmica del modelo anterior

In [ ]:
df_train['back_linear_est'] = np.exp(df_train['log_linear_est'])
df_test['back_linear_est'] = np.exp(df_test['log_linear_est'])

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
plot_time_series(df_train, df_test, 'back_linear_est')

#### Se comparan los valores de RMSE y MAPE del Back Transformation sin y con el agregado de los meses

A pesar de que Back Transformation con y sin Estimate parecen idénticos, hay unas mínimas diferencias en los decimales.

In [ ]:
print("El RMSE de Back Transformation es ", RMSE(df_test['Close'], df_test['back_linear']), ", mientras que el de Back Transformation + Estacionalidad es ", RMSE(df_test['Close'], df_test['back_linear_est']), ".\n La diferencia es de", (RMSE(df_test['Close'], df_test['back_linear']) - RMSE(df_test['Close'], df_test['back_linear_est'])))
print("\nEl MAPE de Back Transformation es ", MAPE(df_test['Close'], df_test['back_linear']), ", mientras que el de Back Transformation + Estacionalidad es ", MAPE(df_test['Close'], df_test['back_linear_est']), ".\n La diferencia es de", (MAPE(df_test['Close'], df_test['back_linear']) - MAPE(df_test['Close'], df_test['back_linear_est'])))

Por ser ínfima la diferencia, no se almacena el valor del modelo con Estimate incluido.

## f) Simple Smoothing

Se aplica Cross Validation para averiguar el nivel óptimo de Simple Smoothing del train data.

In [ ]:
# Se estandarizan los datos
scaler = StandardScaler()
values_standardized = scaler.fit_transform(df_train['Close'].values.reshape(-1, 1)).flatten()

# Se define el rango de hiperparametros a teastear
hyperparam_range = np.linspace(0.001, 1, num=100)

# Se calcula el error de cada hiperparámetro utilizando CV
tscv = TimeSeriesSplit(n_splits=5)
mse_errors = []
for alpha in hyperparam_range:
    errors = []
    for train, test in tscv.split(values_standardized):
        model = SimpleExpSmoothing(values_standardized[train]).fit(smoothing_level=alpha, optimized=False)
        predictions_standardized = model.forecast(len(test))
        actual_standardized = values_standardized[test]
        predictions = scaler.inverse_transform(predictions_standardized.reshape(-1, 1)).flatten()
        actual = scaler.inverse_transform(actual_standardized.reshape(-1, 1)).flatten()
        error = mean_squared_error(predictions, actual)
        errors.append(error)
    mse_errors.append(np.mean(np.array(errors)))

# Se encuentra el hiperparámetro óptimo
optimal_alpha = hyperparam_range[np.argmin(mse_errors)]
print('Optimal alpha:', optimal_alpha)

#### Se fitean varios modelos

Se realizará el proceso 3 veces para comparar los resultados en test. El primer caso será uno sin suavizado: en ese caso, Simple Smoothing equivale a un modelo Standard Naive (por lo que se espera que el resultado sea el mismo que el que se obtuvo aplicando Random Walk). Los otros dos serán con distintos grados de suavizado, siendo uno el obtenido mediante Cross Validation.

In [ ]:
model_no_smoothing = SimpleExpSmoothing(df_train['Close']).fit(smoothing_level = 1, optimized = False)
df_train['standard_naive'] = model_no_smoothing.fittedvalues
df_test['standard_naive'] = model_no_smoothing.forecast(len(df_test))

model_simple_smoothing = SimpleExpSmoothing(df_train['Close']).fit(smoothing_level = 0.1, optimized = False)
df_train['simple_smoothing'] = model_simple_smoothing.fittedvalues
df_test['simple_smoothing'] = model_simple_smoothing.forecast(len(df_test))

model_strong_simple_smoothing = SimpleExpSmoothing(df_train['Close']).fit(smoothing_level = optimal_alpha, optimized = False)
df_train['strong_simple_smoothing'] = model_strong_simple_smoothing.fittedvalues
df_test['strong_simple_smoothing'] = model_strong_simple_smoothing.forecast(len(df_test))

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
plot_time_series(df_train, df_test, 'standard_naive')
plot_time_series(df_train, df_test, 'simple_smoothing')
plot_time_series(df_train, df_test, 'strong_simple_smoothing')

Como se puede observar, el resultado obtenido utilizando el alpha que arroja el cross validation sobre train presenta underfitting, ya que da un rendimiento inferior en comparación con un alpha superior. En otras palabras, el mayor suavizado es menos eficiente que no suavizar (Standard Naive), y mucho menos que un suavizado leve. Esto es razonable en casos como el del presente dataset: los patrones históricos de 2014 a 2017 tienen poca relevancia para predecir movimientos actuales en el precio, en comparación a tendencias recientes como lo son las del 2020 con el boom de las criptomonedas. Un alpha más alto, con menor suavizado y mayor overfitting, podría capturar mejor las fluctuaciones a corto plazo que son más significativas en mercados criptográficos en constante evolución.

#### Se almacenan los valores de RMSE y MAPE

In [ ]:
df_results.loc[4, "Model"] = "Simple Smoothing"
df_results.loc[4, "RMSE"] = round(RMSE(df_test['Close'], df_test['simple_smoothing']),1)
df_results.loc[4, "MAPE"] = round(MAPE(df_test['Close'], df_test['simple_smoothing']),1)
df_results

## g) ARIMA

In [ ]:
stepwise_fit = auto_arima(df_train['Close'], trace = True, suppress_warnings = True)
model_ARIMA = ARIMA(df_train['Close'], order = stepwise_fit.order)

In [ ]:
df_train['arima'] = model_ARIMA.fit().fittedvalues

In [ ]:
forecast_ARIMA = model_ARIMA.fit().get_forecast(steps=len(df_test))
df_test['arima'] = forecast_ARIMA.predicted_mean.values

In [ ]:
plot_time_series(df_train, df_test, 'arima')

#### Se observa el summary:

In [ ]:
print(model_ARIMA.fit().summary())

#### Se almacenan los valores de RMSE y MAPE

In [ ]:
df_results.loc[5, "Model"] = "ARIMA"
df_results.loc[5, "RMSE"] = round(RMSE(df_test['Close'], df_test['arima']),1)
df_results.loc[5, "MAPE"] = round(MAPE(df_test['Close'], df_test['arima']),1)
df_results

## h) Prophet

#### Se busca la mejor combinación de hiperparámetros:

Prophet requiere que la columna Date se llame "ds" y la columna de los precios "y".

In [ ]:
df_train['ds'] = df_train['Date']
df_test['ds'] = df_test['Date']
df_train['y'] = df_train['Close']
df_test['y'] = df_test['Close']

In [ ]:
# Este código toma mucho tiempo. Dependiendo del poder de procesamiento de la computadora, puede demorar alrededor de una hora.

# Como el proceso produce más de 100 líneas de output por cada iteración de prueba de hiperparámetros, se lo reduce via logging:
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)
# De esta manera, solo se produce la barra de progreso de cada iteración.

# Se crea una grilla con distintos valores de parámetros posibles a testear
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.005, 0.01, 0.1, 0.5, 1],
    'seasonality_prior_scale': [1, 10, 20, 30, 40],
    'seasonality_mode' : ('additive', 'multiplicative'),
    'daily_seasonality' : [False, True]}

# Genera todas las combinaciones de parámetros
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []

# Usa cross validation para evaluar los parámetros
for params in all_params:
    m = Prophet(**params).fit(df_train)  # Fitea el modelo con los parámetros obtenidos
    df_cv = cross_validation(m, initial='2500 days', period= '15 days', horizon='31 days')
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

#### Análisis de cuál tuvo mejor rendimiento:

In [ ]:
# Se convierten los resultados en un df
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses

# Se encuentran los mejores hiperparámetros
best_params = tuning_results.loc[tuning_results['rmse'].idxmin()]
print("Best parameters:")
print(best_params)

#### Aplicación de los hiperparámetros sobre el modelo y fiteo

In [ ]:
# Se crea el modelo con los mejores parámetros obtenidos
prophet_model = Prophet(changepoint_prior_scale = best_params['changepoint_prior_scale'],
                        seasonality_prior_scale = best_params['seasonality_prior_scale'],
                        seasonality_mode = best_params['seasonality_mode'],
                        daily_seasonality = best_params['daily_seasonality'])

# Se fitea el modelo
prophet_model.fit(df_train)

# Se crea un Dataframe para realizar las predicciones
future = prophet_model.make_future_dataframe(periods=len(df_test), freq='D')

# Se realizan las predicciones
forecast = prophet_model.predict(future)

In [ ]:
# Se insertan los resultados en el dataset utilizando el mismo esquema que el de los anteriores modelos
forecast_values = forecast['yhat'].values
df_train['prophet'] = forecast_values[:len(df_train)]
df_test['prophet'] = forecast_values[-len(df_test):]

In [ ]:
plot_time_series(df_train, df_test, "prophet")

#### Se almacenan los valores de RMSE y MAPE

In [ ]:
df_results.loc[6, "Model"] = "Prophet"
df_results.loc[6, "RMSE"] = round(RMSE(df_test['Close'], df_test['prophet']),1)
df_results.loc[6, "MAPE"] = round(MAPE(df_test['Close'], df_test['prophet']),1)
df_results

# 3) Comparación de resultados

#### Análisis de RMSE y MAPE visualizado

In [ ]:
fig, ax1 = plt.subplots(figsize = (22, 13))
ax1.set_xlabel('Modelos', fontsize = 22)
ax1.set_ylabel('MAPE', fontsize = 20, color = 'b')
ax1.bar(df_results.index - 0.2, df_results.MAPE, width = 0.4, color = 'b', linewidth = 2, label = "MAPE")
ax1.tick_params(axis = 'y', labelcolor = 'b', labelsize = 17)
ax1.tick_params(axis = 'x', labelsize = 15)
ax1.set_ylim([0, 99])

ax2 = ax1.twinx()
ax2.set_ylabel('RMSE', fontsize = 20, color = 'r')
ax2.bar(df_results.index + 0.2, df_results.RMSE, width = 0.4, color = 'r', linewidth = 2, label = "RMSE")
ax2.tick_params(axis = 'y', labelcolor = 'r', labelsize = 17)
ax2.set_ylim([0, 50000])

plt.axvline(x = 'Mean', color = 'grey', linestyle = '--', lw = 1.3)
plt.axvline(x = 'Random Walk',color = 'grey', linestyle = '--', lw = 1.3)
plt.axvline(x = 'Linear Trend', color = 'grey', linestyle = '--', lw = 1.3)
plt.axvline(x = 'Back Log Linear Trend' , color = 'grey', linestyle = '--', lw = 1.3)
plt.axvline(x = 'Simple Smoothing', color = 'grey', linestyle = '--', lw = 1.3)
plt.axvline(x = 'ARIMA', color = 'grey', linestyle = '--', lw = 1.3)
plt.axvline(x = 'Prophet', color = 'grey', linestyle = '--', lw = 1.3)

plt.grid(which = 'major', axis = 'y', color = 'black', lw = 0.4, alpha = 0.6)
plt.suptitle("Comparación de resultados", fontsize = 24, y = 0.94)
legend1 = ax1.legend(loc = (0.86, 0.9), fontsize = 18)
legend2 = ax2.legend(loc = (0.86, 0.82), fontsize = 18)

plt.show()

# 3) Análisis de estacionalidad y autocorrelación

A continuación, se analizarán ACF, PACF y Dickey Fuller sobre la serie de tiempo de BTC y sobre los residuos de algunos modelos

#### Se crea una función para plotear una serie con información sobre ACF, PACF y su estacionalidad:

In [ ]:
def tsplot(y, model_name = None, lags = None, figsize = (12, 7), style = 'bmh'):
    """
    Plotea la serie de tiempo, el ACF y PACF y el test de Dickey–Fuller
    
    y - serie de tiempo
    model_name - nombre del modelo con default None para cuando se desee plotear la serie de tiempo de BTC, en vez de los residuos del modelo
    lags - cuántos lags incluir para el cálculo de la ACF y PACF
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    with plt.style.context(style):
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)

        # Se definen ejes
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))

        y.plot(ax=ts_ax)

        # Se obtiene el p-value con H0: raiz unitaria presente
        result = sm.tsa.stattools.adfuller(y)
        p_value = result[1]

        if model_name is not None:
            ts_ax.set_title(f"Análisis de los residuos del modelo {model_name}", fontsize=18)
        else:
            ts_ax.set_title("Análisis de la serie de tiempo de BTC", fontsize=18)

        # Se agrega el texto del Dickey Fuller
        adf_text = f"ADF Statistic: {round(result[0],2)}\n"
        adf_text += f"p-value: {round(result[1],4)}"

        # Se añade el texto del Dickey Fuller como anotación
        annotation_box = dict(boxstyle='square,pad=0.5', facecolor='white', edgecolor='black', alpha=1)
        annotation = ts_ax.annotate(adf_text, xy=(0.11, 0.75), xycoords='axes fraction', ha='center', fontsize=12, bbox=annotation_box)

        # Se añade un cuadro para el texto de Dickey Fuller
        annotation_bbox = annotation.get_bbox_patch()
        annotation_bbox.set_boxstyle("round,pad=0.3", pad=0.5)

        # Plot de autocorrelacion
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        # Plot de autocorrelacion parcial
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()

## Serie de BTC

In [ ]:
tsplot(df_train['Close'], lags = 36)

## Residuos

#### Se crea la variable de cada residuo

In [ ]:
residue_mean = df_train['Close'] - df_train['mean']
residue_random_walk = df_train['Close'] - df_train['random_walk']
residue_linear_trend = df_train['Close'] - df_train['linear_trend']
residue_back_linear = df_train['Close'] - df_train['back_linear']
residue_simple_smoothing = df_train['Close'] - df_train['simple_smoothing']
residue_arima = df_train['Close'] - df_train['arima']
residue_prophet = df_train['Close'] - df_train['prophet']

#### Se plotean todos los residuos

In [ ]:
residues = [residue_mean, residue_random_walk, residue_linear_trend, residue_back_linear, residue_simple_smoothing, residue_arima, residue_prophet]

for residue, name in zip(residues, ["Mean", "Random Walk", "Linear Trend", "Back Linear", "Simple Smoothing", "ARIMA", "Prophet"]):
    tsplot(residue, model_name=name, lags=36)

## Análisis

#### Serie de tiempo de BTC

La estadística ADF de 0.7, junto con un p value de 0.99, indican en conjunto que la prueba de Dickey-Fuller Aumentada no logra proporcionar una evidencia sólida en contra de la presencia de una raíz unitaria en la serie de tiempo del BTC. Esto sugiere que los datos siguen siendo no estacionarios, potencialmente mostrando tendencias y patrones que pueden afectar el análisis y la predicción.

#### Residuos de modelos

Algunos valores de p value y de estadística de ADF son resultados esperados:

1) Los valores para los residuos de Mean son idénticos a los de la serie de BTC.
2) Los valores para los residuos de Random Walk, Simple Smoothing con poco suavizado y ARIMA en la configuración elegida dan estacionariedad perfecta. Esto se debe a que los valores que los modelos proponen para train son sumamente similares a los actuales de train.

Algunas conclusiones que se pueden realizar sobre otros modelos son:

1) Linear Trend
    - Estadística ADF: -1.36
    - Valor p: 0.6
    - Análisis: La estadística ADF de -1.36 sugiere no estacionariedad, ya que no es fuertemente negativa. El valor p alto de 0.6 indica que no hay suficiente evidencia para rechazar la hipótesis nula de una raíz unitaria, lo que respalda la idea de no estacionariedad.


2) Back Log Linear Trend
    - Estadística ADF: -2.84
    - Valor p: 0.05
    - Análisis: La estadística ADF de -2.84 es más negativa, lo que sugiere una evidencia más fuerte en contra de la presencia de una raíz unitaria e indica una mayor probabilidad de estacionariedad. El valor p de 0.05 es relativamente bajo, lo que indica que existe alguna evidencia para rechazar la hipótesis nula de una raíz unitaria, lo cual concuerda con la estadística ADF que sugiere potencial estacionariedad.


3) Prophet
    - Estadística ADF: -4.13
    - Valor p: 0.0009
    - Análisis: La estadística ADF muy baja de -4.13 sugiere una evidencia sólida en contra de la presencia de una raíz unitaria e indica una alta probabilidad de estacionariedad. El valor p muy bajo de 0.0009 confirma esta evidencia, rechazando fuertemente la hipótesis nula de una raíz unitaria y respaldando la conclusión de estacionariedad.